In [ ]:
import torch.optim as optim
from torch.nn import TripletMarginLoss
from torch.utils.data import DataLoader
from dataloader import RandomTripletLossDataset, collate_triplet_fn
from Frontend import MFCCTransform
from Model import SpeakerClassifier
import utils

In [ ]:
MFCCS = 13

MODEL = "MFCC_ECAPA-TDNN_Genuine_Random"

LEARNING_RATE = 0.001
MARGIN = 1
NORM = 2

BATCH_SIZE = 32
EPOCHS = 2

In [ ]:
logger = utils.initialize_environment(MODEL)
device = utils.get_device()

In [ ]:
train_labels, dev_labels, test_labels = utils.load_genuine_dataset()

In [ ]:
audio_dataset = RandomTripletLossDataset(train_labels, frontend=MFCCTransform)
audio_dataloader = DataLoader(audio_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_triplet_fn)
model = SpeakerClassifier(input_size=MFCCS, device=device)
model.to(device)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
triplet_loss = TripletMarginLoss(margin=MARGIN, p=NORM)

In [ ]:
# Run training
utils.train_model_random_loss(EPOCHS, audio_dataloader, model, triplet_loss, optimizer, device)